In [ ]:
import socket
import time
import math

# 닉네임을 사용자에 맞게 변경해 주세요.
NICKNAME = 'GWANGJU01_YEODOHYEON'

# 일타싸피 프로그램을 로컬에서 실행할 경우 변경하지 않습니다.
HOST = '127.0.0.1'

# 일타싸피 프로그램과 통신할 때 사용하는 코드값으로 변경하지 않습니다.
PORT = 1447
CODE_SEND = 9901
CODE_REQUEST = 9902
SIGNAL_ORDER = 9908
SIGNAL_CLOSE = 9909


# 게임 환경에 대한 상수입니다.
TABLE_WIDTH = 254
TABLE_HEIGHT = 127
NUMBER_OF_BALLS = 6
HOLES = [[0, 0], [127, 0], [254, 0], [0, 127], [127, 127], [254, 127]]

order = 0
balls = [[0, 0] for i in range(NUMBER_OF_BALLS)]

sock = socket.socket()
print('Trying to Connect: %s:%d' % (HOST, PORT))
sock.connect((HOST, PORT))
print('Connected: %s:%d' % (HOST, PORT))

send_data = '%d/%s' % (CODE_SEND, NICKNAME)
sock.send(send_data.encode('utf-8'))
print('Ready to play!\n--------------------')


while True:

    # Receive Data
    recv_data = (sock.recv(1024)).decode()
    print('Data Received: %s' % recv_data)

    # Read Game Data
    split_data = recv_data.split('/')
    idx = 0
    try:
        for i in range(NUMBER_OF_BALLS):
            for j in range(2):
                balls[i][j] = float(split_data[idx])
                idx += 1
    except:
        send_data = '%d/%s' % (CODE_REQUEST, NICKNAME)
        print("Received Data has been currupted, Resend Requested.")
        continue

    # Check Signal for Player Order or Close Connection
    if balls[0][0] == SIGNAL_ORDER:
        order = int(balls[0][1])
        print('\n* You will be the %s player. *\n' % ('first' if order == 1 else 'second'))
        continue
    elif balls[0][0] == SIGNAL_CLOSE:
        break

    # Show Balls' Position
    print('====== Arrays ======')
    for i in range(NUMBER_OF_BALLS):
        print('Ball %d: %f, %f' % (i, balls[i][0], balls[i][1]))
    print('====================')

    angle = 0.0
    power = 0.0

    ##############################
    # 이 위는 일타싸피와 통신하여 데이터를 주고 받기 위해 작성된 부분이므로 수정하면 안됩니다.
    #
    # 모든 수신값은 변수, 배열에서 확인할 수 있습니다.
    #   - order: 1인 경우 선공, 2인 경우 후공을 의미
    #   - balls[][]: 일타싸피 정보를 수신해서 각 공의 좌표를 배열로 저장
    #     예) balls[0][0]: 흰 공의 X좌표
    #         balls[0][1]: 흰 공의 Y좌표
    #         balls[1][0]: 1번 공의 X좌표
    #         balls[4][0]: 4번 공의 X좌표
    #         balls[5][0]: 마지막 번호(8번) 공의 X좌표

    # 여기서부터 코드를 작성하세요.
    # 아래에 있는 것은 샘플로 작성된 코드이므로 자유롭게 변경할 수 있습니다.

'''
a = sqrt((xh-))
d = sqrt(a^2 + (b+2*r)^2 - 2a*(b+2*r)*cos(다))
나 = acos((a^2 + d^2 -(b+2*r)^2)/(2*a*d))
degree_나 = math.degrees(나)
가 = atan(x/y)
degree_가 = math.degrees(가)
theta = degree_가 + degree_나
theta_shoot = 90 - theta
'''

# 여기서부터 코드를 작성하세요.
    # 아래에 있는 것은 샘플로 작성된 코드이므로 자유롭게 변경할 수 있습니다.

    r = 5.73  # 공의 반지름
    whiteBall_x = balls[0][0]
    whiteBall_y = balls[0][1]
    whiteBall_pos = [whiteBall_x, whiteBall_y]

    # 1. 내가 쳐야 할 공 목록 정의
    if order == 1:
        my_balls_indices = [1, 3, 5]  # 선공일 때 쳐야 할 공 인덱스
    else:
        my_balls_indices = [2, 4, 5]  # 후공일 때 쳐야 할 공 인덱스

    # 2. 샷을 찾기 위한 변수 초기화
    best_shot_score = 99999  # 가장 좋은 점수를 저장할 변수 
    final_angle = 0
    final_power = 0

    # 3. 모든 목표 공과 모든 홀의 조합을 순회
    for ball_idx in my_balls_indices:
        
        # 만약 해당 공이 이미 들어갔다면 건너뛰기
        if balls[ball_idx][0] == -1:
            continue
        
        targetBall_pos = balls[ball_idx]

        for hole_pos in HOLES:
            
            # 4. 현재 조합(목표 공, 홀)에 대한 정보 계산
            
            # 4-1. 목표 공을 홀에 넣기 위해 흰 공이 맞춰야 할 가상 지점(고스트볼) 계산
            # 벡터 계산
            vec_x = targetBall_pos[0] - hole_pos[0]
            vec_y = targetBall_pos[1] - hole_pos[1]
            vec_mag = math.sqrt(vec_x**2 + vec_y**2)
            
            # 단위 벡터
            unit_vec_x = vec_x / vec_mag
            unit_vec_y = vec_y / vec_mag
            
            # 고스트볼 위치 (목표 공 중심에서 공 지름만큼 떨어진 곳)
            ghostBall_x = targetBall_pos[0] + unit_vec_x * (2 * r)
            ghostBall_y = targetBall_pos[1] + unit_vec_y * (2 * r)
            ghostBall_pos = [ghostBall_x, ghostBall_y]
            
            # 4-2. 샷의 난이도를 평가할 지표 계산
            # - 샷의 각도 (컷 각도): 0에 가까울수록 직선 샷
            # - 필요한 거리 등
            
            # 흰 공 -> 고스트볼 벡터
            v_wg_x = ghostBall_x - whiteBall_x
            v_wg_y = ghostBall_y - whiteBall_y
            
            # 목표 공 -> 홀 벡터
            v_th_x = hole_pos[0] - targetBall_pos[0]
            v_th_y = hole_pos[1] - targetBall_pos[1]
            
            # 두 벡터 사이의 각도(컷 각도) 계산
            dot_product = v_wg_x * v_th_x + v_wg_y * v_th_y
            mag_wg = math.sqrt(v_wg_x**2 + v_wg_y**2)
            mag_th = math.sqrt(v_th_y**2 + v_th_x**2)
            
            # cos(theta) 값이므로, acos을 통해 라디안을 구하고 각도로 변환
            # ZeroDivisionError 방지
            if mag_wg == 0 or mag_th == 0:
                continue
            cos_theta = dot_product / (mag_wg * mag_th)
            # 부동소수점 오류로 가끔 1.0을 초과하거나 -1.0 미만이 될 수 있어 clamp 처리
            cos_theta = max(-1.0, min(1.0, cos_theta))
            cut_angle = math.degrees(math.acos(cos_theta))

            # 4-3. 불가능한 샷 필터링 (흰 공이 목표 공보다 앞에 있을 때)
            dist_white_target = math.sqrt((targetBall_pos[0] - whiteBall_x)**2 + (targetBall_pos[1] - whiteBall_y)**2)
            dist_white_ghost = mag_wg
            if dist_white_target < r * 2: # 너무 가까운 공은 제외
                continue
            if dist_white_ghost < dist_white_target: # 샷이 불가능한 방향일 때
                continue
            
            # 5. 현재 샷의 '점수' 계산 (점수가 낮을수록 좋은 샷)
            # 컷 각도가 클수록, 거리가 멀수록 어려운 샷으로 평가
            score = cut_angle * 1.2 + dist_white_ghost * 0.3 + mag_th * 0.3

            # 6. '최고의 샷' 정보 업데이트
            # 계산된 점수가 현재까지의 최고 점수보다 좋다면, 이 샷을 '최고의 샷'으로 임시 저장
            if score < best_shot_score:
                best_shot_score = score
                
                # 최종적으로 사용할 각도 계산 (atan2 사용)
                radian = math.atan2(ghostBall_y - whiteBall_y, ghostBall_x - whiteBall_x)
                final_angle = math.degrees(radian) 
                
                # 최종적으로 사용할 파워 계산
                final_power = (dist_white_ghost + mag_th) * 0.4
    
    # 순회가 끝난 후 저장된 '최고의 샷' 정보로 angle, power 변수에 할당
    angle = final_angle
    power = final_power
    
    # 힘이 100을 넘지 않도록 조절
    if power > 100:
        power = 100
    
    # 만약 칠 공이 하나도 없다면 (my_balls_indices가 비어있거나 모든 공이 들어간 경우)
    # 안전하게 약한 샷을 하거나 다른 방어적인 플레이를 할 수 있습니다.
    # 여기서는 간단히 angle/power가 0으로 남아있게 됩니다.

    ##############################
    # 이 위는 일타싸피와 통신하여 데이터를 주고 받기 위해 작성된 부분이므로 수정하면 안됩니다.


In [ ]:

    r = 5.73  # 공의 반지름
    whiteBall_x = balls[0][0]
    whiteBall_y = balls[0][1]
    whiteBall_pos = [whiteBall_x, whiteBall_y]

    # 1. 내가 쳐야 할 공 목록 정의
    if order == 1:
        my_balls_indices = [1, 3, 5]
    else:
        my_balls_indices = [2, 4, 5]

    # 2. 가장 좋은 샷을 찾기 위한 변수 초기화
    # 이번 로직에서는 가장 가까운 공을 치는 것을 '최고의 샷'으로 간주합니다.
    best_shot_distance = 99999
    final_angle = 0
    final_power = 0
    
    # 3. 모든 홀을 기준으로 순회 시작
    for hole_pos in HOLES:
        
        # 3-1. '흰 공 -> 홀' 벡터와 거리 계산
        vec_wh_x = hole_pos[0] - whiteBall_x
        vec_wh_y = hole_pos[1] - whiteBall_y
        dist_wh = math.sqrt(vec_wh_x**2 + vec_wh_y**2)
        
        # 3-2. '흰 공 -> 홀' 방향의 절대 각도 계산 (기준선)
        angle_wh_rad = math.atan2(vec_wh_y, vec_wh_x)

        # 4. 쳐야 할 공들을 순회하며 조건 확인
        for ball_idx in my_balls_indices:
            
            # 이미 들어간 공은 건너뛰기
            if balls[ball_idx][0] == -1:
                continue
            
            targetBall_pos = balls[ball_idx]
            
            # 4-1. '흰 공 -> 목표 공' 벡터와 거리 계산
            vec_wt_x = targetBall_pos[0] - whiteBall_x
            vec_wt_y = targetBall_pos[1] - whiteBall_y
            dist_wt = math.sqrt(vec_wt_x**2 + vec_wt_y**2)

            # 4-2. '흰 공 -> 목표 공' 방향의 절대 각도 계산
            angle_wt_rad = math.atan2(vec_wt_y, vec_wt_x)

            # 5. 로직의 핵심 조건 검사
            
            # 조건 1: 두 방향 벡터 사이의 각도 차이가 30도 이내인가?
            angle_diff_deg = abs(math.degrees(angle_wh_rad) - math.degrees(angle_wt_rad))
            # 각도 차이가 180도를 넘어가면 반대 방향으로 계산
            if angle_diff_deg > 180:
                angle_diff_deg = 360 - angle_diff_deg
            
            # 조건 2: 목표 공이 흰 공과 홀 '사이'에 있는가? (홀보다 멀리 있으면 안 됨)
            is_between = dist_wt < dist_wh
            
            # 5-1. 두 조건을 모두 만족하는가?
            if angle_diff_deg <= 30 and is_between:
                # 6. 조건을 만족하는 공들 중 가장 가까운 공을 '최고의 샷'으로 선택
                if dist_wt < best_shot_distance:
                    best_shot_distance = dist_wt
                    
                    # 최종 각도는 '흰 공 -> 목표 공'을 직접 조준
                    final_angle = math.degrees(angle_wt_rad)
                    
                    # 최종 파워는 흰 공->목표 공, 목표 공->홀 거리를 합산해 계산
                    dist_th = math.sqrt((hole_pos[0] - targetBall_pos[0])**2 + (hole_pos[1] - targetBall_pos[1])**2)
                    final_power = (dist_wt + dist_th) * 0.5 # 0.5는 세기 조절을 위한 계수

    # 7. 순회가 끝난 후 찾은 최고의 샷 정보로 angle, power 설정
    angle = final_angle
    power = final_power

    # 힘이 100을 넘지 않도록 조절
    if power > 100:
        power = 100
        
    ##############################

In [ ]:
r = 5.73
whiteBall_x = balls[0][0]
whiteBall_y = balls[0][1]
whiteBall_pos = [whiteBall_x, whiteBall_y]

if order == 1:
    my_balls = [1, 3, 5]
else:
    my_balls = [2, 4, 5]

best_shot_distance = 99999
final_angle = 0
final_power = 0

# 모든 홀을 기준으로 순회
for hole_pos in HOLES:

    wh_x = hole_pos[0] - whiteBall_x
    wh_y = hole_pos[1] - whiteBall_y
    dist_wh = math.sqrt(wh_x**2 + wh_y**2)

    #흰공 -> 홀 방향의 각도 계산
    angle_wh_rad = math.atan2(wh_y, wh_x)

In [ ]:
# 여기서부터 코드를 작성하세요.
    # 리스트와 min() 함수를 사용하여 최적의 샷을 찾는 최종 코드입니다.

    r = 5.73  # 공의 반지름
    whiteBall_x = balls[0][0]
    whiteBall_y = balls[0][1]

    # 1. 내가 쳐야 할 공 목록 정의
    if order == 1:
        my_balls_indices = [1, 3, 5]
    else:
        my_balls_indices = [2, 4, 5]

    # 2. 가능한 모든 유효한 샷의 정보를 저장할 리스트 생성
    candidate_shots = []

    # 3. 모든 (목표 공, 홀) 조합을 순회하며 샷 정보 수집
    for ball_idx in my_balls_indices:
        if balls[ball_idx][0] == -1: continue
        
        targetBall_pos = balls[ball_idx]

        for hole_pos in HOLES:
            # 3-1. 컷 각도 계산 (atan2 방식)
            vec_th_x = targetBall_pos[0] - hole_pos[0]
            vec_th_y = targetBall_pos[1] - hole_pos[1]
            mag_th = math.sqrt(vec_th_x**2 + vec_th_y**2)
            if mag_th == 0: continue

            ghostBall_x = targetBall_pos[0] + (vec_th_x / mag_th) * (2 * r)
            ghostBall_y = targetBall_pos[1] + (vec_th_y / mag_th) * (2 * r)

            vec_wg_x = ghostBall_x - whiteBall_x
            vec_wg_y = ghostBall_y - whiteBall_y
            mag_wg = math.sqrt(vec_wg_x**2 + vec_wg_y**2)

            # 불가능한 샷 필터링 (중요)
            dist_white_target = math.sqrt((targetBall_pos[0] - whiteBall_x)**2 + (targetBall_pos[1] - whiteBall_y)**2)
            if mag_wg <= dist_white_target:
                continue # 칠 수 없는 방향이면 이 샷은 후보에서 제외

            # 각 벡터의 절대 각도를 atan2로 계산
            angle_wg_rad = math.atan2(vec_wg_y, vec_wg_x)
            angle_th_rad = math.atan2(vec_th_y, vec_th_x)

            # 두 절대 각도의 차이를 구해서 컷 각도 계산
            cut_angle = abs(math.degrees(angle_wg_rad) - math.degrees(angle_th_rad))
            if cut_angle > 180:
                cut_angle = 360 - cut_angle

            # 3-2. 유효한 샷 정보를 리스트에 추가
            # (컷 각도, 목표공 위치, 홀 위치) 순서로 저장
            candidate_shots.append([cut_angle, targetBall_pos, hole_pos])
    
    # 4. 모든 후보 샷 중에서 최고의 샷 선택
    angle = 0
    power = 0
    if candidate_shots: # 후보 샷이 하나라도 있다면
        # min() 함수가 리스트의 첫 번째 요소(컷 각도)를 기준으로 가장 작은 값을 찾아줌
        best_shot = min(candidate_shots)
        
        # 최고의 샷 정보 추출
        best_cut_angle = best_shot[0]
        target_pos = best_shot[1]
        hole_pos = best_shot[2]

        # 5. 최종 angle, power 계산
        vec_x = target_pos[0] - hole_pos[0]
        vec_y = target_pos[1] - hole_pos[1]
        vec_mag = math.sqrt(vec_x**2 + vec_y**2)
        if vec_mag == 0: vec_mag = 1

        unit_vec_x = vec_x / vec_mag
        unit_vec_y = vec_y / vec_mag
        
        ghostBall_x = target_pos[0] + unit_vec_x * (2 * r)
        ghostBall_y = target_pos[1] + unit_vec_y * (2 * r)

        angle = math.degrees(math.atan2(ghostBall_y - whiteBall_y, ghostBall_x - whiteBall_x))
        
        dist_white_ghost = math.sqrt((ghostBall_x - whiteBall_x)**2 + (ghostBall_y - whiteBall_y)**2)
        power = (dist_white_ghost + vec_mag) * 0.4

    # 힘이 100을 넘지 않도록 조절
    if power > 100:
        power = 100
        
    ##############################